In [1]:
# If we aren't on a c3 container, we need to get the c3 object.
try:
    c3
except:
    from c3python import get_c3
    c3 = get_c3('http://192.168.2.44:8080', 'mnistExercise', 'dev')

Username: BA
Password: ········


In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# Load and 'reshape' datasets
#mnist_test_images = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Test_Images').getDataset(),
#                                         axesAsRow=[0], axesAsColumn=[1,2])
#mnist_test_images.axes = ['source', 'features']
#mnist_test_labels = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Test_Labels').getDataset(),
#                                         axesAsRow=[0], axesAsColumn=[1])
#mnist_test_labels.axes = ['source', 'features']
#mnist_train_images = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Train_Images').getDataset(),
#                                          axesAsRow=[0], axesAsColumn=[1,2])
#mnist_train_images.axes = ['source', 'features']
#mnist_train_labels = c3.Dataset.toDataset(this=c3.IDXFile.get('MNIST_Train_Labels').getDataset(),
#                                          axesAsRow=[0], axesAsColumn=[1])
#mnist_train_labels.axes = ['source', 'features']

mnist_test_images = c3.IDXFile.get('MNIST_Test_Images').getDataset()
mnist_test_labels = c3.IDXFile.get('MNIST_Test_Labels').getDataset()
mnist_train_images = c3.IDXFile.get('MNIST_Train_Images').getDataset()
mnist_train_labels = c3.IDXFile.get('MNIST_Train_Labels').getDataset()

In [4]:
mnist_test_labels_onehot = mnist_test_labels.generateDummyVariables(column='0', keepOriginalColumn=False)
mnist_train_labels_onehot = mnist_train_labels.generateDummyVariables(column='0', keepOriginalColumn=False)

In [8]:
def train_and_test_model(name, model, train_X, train_Y, test_X, test_Y, epochs=10, callbacks=[]):
    pipe = c3.KerasPipe().upsertNativeModel(model=model)
    pipe.technique.numEpochs = epochs
    trained_pipe = pipe.train(input=train_X, targetOutput=train_Y)
    result = trained_pipe.process(input=test_X)

    result_np = result.toNumpyArray()
    test_Y_np = test_Y.toNumpyArray()

    tot = test_Y_np.shape[0]
    error_rate = sum(result_np.argmax(axis=1)!=test_Y_np.argmax(axis=1))/(float(tot))

    print("{} Error Rate: {:.2f}%".format(name, error_rate*100))
    return trained_pipe

In [9]:
# Build tensorflow model.
one_layer_simple_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28*28,), input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

one_layer_simple_model.summary()
one_layer_simple_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 795,010
Trainable params: 795,010
Non-trainable params: 0
_________________________________________________________________


In [10]:
train_and_test_model('one_layer_simple_model', one_layer_simple_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

one_layer_simple_model Error Rate: 7.60%


c3.KerasPipe(
 id='48b0e5fa-dd8e-46fa-ada3-5e1999d78255',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 43, 19, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 18, 45, 19, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 18, 45, 19, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_1", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_1", "trainable": true, '
                       '"batch_input_

In [11]:
# Build tensorflow model.
two_layer_simple_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28*28,), input_shape=(28,28)),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

two_layer_simple_model.summary()
two_layer_simple_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_6 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [12]:
train_and_test_model('two_layer_simple_model', two_layer_simple_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

two_layer_simple_model Error Rate: 6.50%


c3.KerasPipe(
 id='4e241871-0d82-4a37-82f3-ec898ddb253d',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 48, 44, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 18, 50, 44, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 18, 50, 44, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_2", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_2", "trainable": true, '
                       '"batch_input_

In [13]:
one_layer_cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

one_layer_cnn_model.summary()
one_layer_cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                216330    
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [14]:
train_and_test_model('one_layer_cnn_model', one_layer_cnn_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

one_layer_cnn_model Error Rate: 5.00%


c3.KerasPipe(
 id='6a4f4628-93eb-402b-a42e-38c7a3db837b',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 54, 11, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 18, 56, 8, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 18, 56, 8, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_3", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_3", "trainable": true, '
                       '"batch_input_sh

In [15]:
three_layer_cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.Conv2D(64, kernel_size=(3,3),
                           activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(125, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

three_layer_cnn_model.summary()
three_layer_cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 125)              

In [16]:
train_and_test_model('three_layer_cnn_model', three_layer_cnn_model,
                     mnist_train_images, mnist_train_labels_onehot,
                     mnist_test_images, mnist_test_labels_onehot)

three_layer_cnn_model Error Rate: 2.89%


c3.KerasPipe(
 id='f105bd4e-65e9-4924-aa75-fa7bd7641ae2',
 meta=c3.Meta(
        tenantTagId=3,
        tenant='mnistExercise',
        tag='dev',
        created=datetime.datetime(2020, 6, 19, 18, 59, 32, tzinfo=datetime.timezone.utc),
        createdBy='BA',
        updated=datetime.datetime(2020, 6, 19, 19, 1, 50, tzinfo=datetime.timezone.utc),
        updatedBy='BA',
        timestamp=datetime.datetime(2020, 6, 19, 19, 1, 50, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='KerasPipe'),
 version=2,
 typeIdent='PIPE:LF:KRS',
 implType='KerasPipeV7d12',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 typeVersion='7.12.4',
 untrainableOverride=False,
 technique=c3.KerasTechnique(
             modelDef='{"class_name": "Sequential", "config": {"name": '
                       '"sequential_4", "layers": [{"class_name": "Reshape", '
                       '"config": {"name": "reshape_4", "trainable": true, '
                       '"batch_input_sh